In [1]:
import numpy as np
import pandas as pd
# %matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
from numpy.linalg import solve

np.random.seed(0)

#### we based our work on the popularity recommendation model on the blog post:
#### https://www.datacamp.com/community/tutorials/recommender-systems-python

![title](popularity_formula.png)

In [107]:
# Calculate the dataset sparsity, distribution of number of ratings as well as the average rating value per user\item. 
# Include additional exploration you find relevant to questions 2 and 3.
# Discuss your insights and possible challenges related to the prediction task described in question 2.


# Remember that sparsity is calculated by the number of cells in a matrix that contain a rating divided by the total number of values that matrix could hold given the number of users and items (movies). 
# In other words, dividing the number of ratings present in the matrix by the product of users and movies in the matrix and subtracting that from 1 will give us the sparsity or the percentage of the ratings matrix that is empty.

file_path = 'ml-100k/u.data' # grouplens.org/datasets/movielens/100k

names = ['user_id', 'item_id', 'rating', 'timestamp']
data = pd.read_csv(file_path, sep='\t', names=names)

# Calculate the dataset sparsity, distribution of number of ratings as well as the average rating value per user\item. 
# Include additional exploration you find relevant to questions 2 and 3.
# Discuss your insights and possible challenges related to the prediction task described in question 2.


# Remember that sparsity is calculated by the number of cells in a matrix that contain a rating divided by the total number of values that matrix could hold given the number of users and items (movies). 
# In other words, dividing the number of ratings present in the matrix by the product of users and movies in the matrix and subtracting that from 1 will give us the sparsity or the percentage of the ratings matrix that is empty.

movie_file_path = 'ml-100k/u-utf8.item' # grouplens.org/datasets/movielens/100k
header_str = "item_id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation |Children's | Comedy | Crime | Documentary | Drama | Fantasy |Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |Thriller | War | Western "
header = [h.strip() for h in header_str.split('|')]
print(header)

movie_data = pd.read_csv(movie_file_path, sep='|', names=header)

movie_data_subset = movie_data[['item_id', "movie title"]]

['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [124]:
data

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [38]:
data[['item_id', 'rating']].groupby('item_id').count()

,rating
item_id,
1,452
2,131
3,90
4,209
5,86
...,...
1678,1
1679,1
1680,1


In [109]:


metadata = pd.DataFrame(data={'item_id': data.item_id.unique()})
metadata

,item_id
0,242
1,302
2,377
3,51
4,346
...,...
1677,1674
1678,1640
1679,1637
1680,1630


# avg rating

In [35]:
c = data['rating'].mean()
print(c)

3.52986


# vote count 90 percentile

In [65]:
data_vote_count = data[['item_id', 'rating']].groupby('item_id', as_index=False).count().rename(columns={"rating": "vote_count"})
data_vote_count

,item_id,vote_count
0,1,452
1,2,131
2,3,90
3,4,209
4,5,86
...,...,...
1677,1678,1
1678,1679,1
1679,1680,1
1680,1681,1


In [66]:
data_vote_avg_rating = data[['item_id', 'rating']].groupby('item_id',as_index=False).mean().rename(columns={"rating": "vote_average"})
data_vote_avg_rating

,item_id,vote_average
0,1,3.878319
1,2,3.206107
2,3,3.033333
3,4,3.550239
4,5,3.302326
...,...,...
1677,1678,1.000000
1678,1679,3.000000
1679,1680,2.000000
1680,1681,3.000000


In [111]:
metadata_full = metadata.merge(movie_data_subset, on='item_id').merge(data_vote_count, on='item_id').merge(data_vote_avg_rating, on='item_id')
metadata_full

,item_id,movie title,vote_count,vote_average
0,242,Kolya (1996),117,3.991453
1,302,L.A. Confidential (1997),297,4.161616
2,377,Heavyweights (1994),13,2.153846
3,51,Legends of the Fall (1994),81,3.456790
4,346,Jackie Brown (1997),126,3.642857
...,...,...,...,...
1677,1674,Mamma Roma (1962),1,4.000000
1678,1640,"Eighth Day, The (1996)",1,3.000000
1679,1637,Girls Town (1996),1,3.000000
1680,1630,"Silence of the Palace, The (Saimt el Qusur) (1...",1,3.000000


In [113]:
C = metadata_full['vote_average'].mean()
C

3.0760445083251793

In [114]:
m = metadata_full['vote_count'].quantile(0.90)
m

169.0

In [116]:
q_movies = metadata_full.copy().loc[metadata_full['vote_count'] >= int(m)]
q_movies

,item_id,movie title,vote_count,vote_average
1,302,L.A. Confidential (1997),297,4.161616
5,474,Dr. Strangelove or: How I Learned to Stop Worr...,194,4.252577
6,265,"Hunt for Red October, The (1990)",227,3.863436
8,451,Grease (1978),170,3.347059
10,257,Men in Black (1997),303,3.745875
...,...,...,...,...
751,340,Boogie Nights (1997),189,3.587302
801,684,In the Line of Fire (1993),169,3.792899
860,326,G.I. Jane (1997),175,3.360000
862,628,Sleepers (1996),169,3.514793


In [117]:
metadata_full.shape

(1682, 4)

In [118]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [119]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [120]:
q_movies = q_movies.sort_values('score', ascending=False)

In [122]:
q_movies.head(20)

,item_id,movie title,vote_count,vote_average,score
357,50,Star Wars (1977),583,4.358491,4.070281
200,318,Schindler's List (1993),298,4.466443,3.963279
239,64,"Shawshank Redemption, The (1994)",283,4.445230,3.933300
189,127,"Godfather, The (1972)",413,4.283293,3.932735
31,98,"Silence of the Lambs, The (1991)",390,4.289744,3.922811
101,174,Raiders of the Lost Ark (1981),420,4.252381,3.914858
174,483,Casablanca (1942),243,4.456790,3.890416
49,100,Fargo (1996),508,4.155512,3.886044
297,12,"Usual Suspects, The (1995)",267,4.385768,3.878100
719,313,Titanic (1997),350,4.245714,3.864839
